# Label Studio


>[Label Studio](https://labelstud.io/guide/get_started) 是一个开源数据标注平台，它为 LangChain 在微调大型语言模型（LLM）的数据标注方面提供了灵活性。它还可以用于准备自定义训练数据以及通过人类反馈收集和评估响应。

在本指南中，您将学习如何将 LangChain 管道连接到 `Label Studio` 以便：

- 将所有输入提示、对话和响应聚合到一个 `Label Studio` 项目中。这使得所有数据集中在一个地方，更便于标注和分析。
- 精炼提示和响应，为监督微调（SFT）和人类反馈强化学习（RLHF）场景创建数据集。这些标注数据可用于进一步训练 LLM 以改进其性能。
- 通过人类反馈评估模型响应。`Label Studio` 提供了一个界面，供人类审查和提供模型响应的反馈，从而实现评估和迭代。

## 安装与设置

首先，安装最新版本的 Label Studio 和 Label Studio API 客户端：

In [ ]:
%pip install --upgrade --quiet langchain label-studio label-studio-sdk langchain-openai langchain-community

接下来，在命令行中运行 `label-studio`，以在 `http://localhost:8080` 启动本地 LabelStudio 实例。有关更多选项，请参阅 [Label Studio 安装指南](https://labelstud.io/guide/install)。

您需要一个令牌才能进行 API 调用。

在浏览器中打开您的 LabelStudio 实例，转到 `Account & Settings > Access Token` 并复制密钥。

使用您的 LabelStudio URL、API 密钥和 OpenAI API 密钥设置环境变量：

In [ ]:
import os

os.environ["LABEL_STUDIO_URL"] = "<YOUR-LABEL-STUDIO-URL>"  # e.g. http://localhost:8080
os.environ["LABEL_STUDIO_API_KEY"] = "<YOUR-LABEL-STUDIO-API-KEY>"
os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"

## 收集大型语言模型的提示和响应

Label Studio 中用于标记的数据存储在项目内。每个项目都由一个 XML 配置进行标识，该配置详细说明了输入和输出数据的规范。

创建这样一个项目，它接收文本格式的真人输入，并以文本区域的形式输出可编辑的 LLM 回复：

```xml
<View>
<Style>
    .prompt-box {
        background-color: white;
        border-radius: 10px;
        box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        padding: 20px;
    }
</Style>
<View className="root">
    <View className="prompt-box">
        <Text name="prompt" value="$prompt"/>
    </View>
    <TextArea name="response" toName="prompt"
              maxSubmissions="1" editable="true"
              required="true"/>
</View>
<Header value="Rate the response:"/>
<Rating name="rating" toName="prompt"/>
</View>
```

1. 要在 Label Studio 中创建项目，请点击“创建”按钮。
2. 在“项目名称”字段中输入您的项目名称，例如 `My Project`。
3. 导航至 `Labeling Setup > Custom Template` 并粘贴上面提供的 XML 配置。

您可以通过 `LabelStudioCallbackHandler` 将 LLM 提示和输出响应收集到 LabelStudio 项目中：

In [1]:
from langchain_community.callbacks.labelstudio_callback import (
    LabelStudioCallbackHandler,
)

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0, callbacks=[LabelStudioCallbackHandler(project_name="My Project")]
)
print(llm.invoke("Tell me a joke"))

在 Label Studio 中，打开 `My Project`。您将看到提示、响应以及模型名称等元数据。

## 收集聊天模型对话

您还可以通过 LabelStudio 跟踪和显示完整的聊天对话，并能够评分和修改最后的响应：

1. 打开 Label Studio 并单击“Create”按钮。
2. 在“Project Name”字段中输入项目名称，例如 `New Project with Chat`。
3. 导航到 Labeling Setup > Custom Template 并粘贴以下 XML 配置：

```xml
<View>
<View className="root">
     <Paragraphs name="dialogue"
               value="$prompt"
               layout="dialogue"
               textKey="content"
               nameKey="role"
               granularity="sentence"/>
  <Header value="Final response:"/>
    <TextArea name="response" toName="dialogue"
              maxSubmissions="1" editable="true"
              required="true"/>
</View>
<Header value="Rate the response:"/>
<Rating name="rating" toName="dialogue"/>
</View>
```

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat_llm = ChatOpenAI(
    callbacks=[
        LabelStudioCallbackHandler(
            mode="chat",
            project_name="New Project with Chat",
        )
    ]
)
llm_results = chat_llm.invoke(
    [
        SystemMessage(content="Always use a lot of emojis"),
        HumanMessage(content="Tell me a joke"),
    ]
)

在 Label Studio 中，打开“New Project with Chat”。单击已创建的任务以查看对话历史记录并编辑/注释响应。

## 自定义标签配置

Here's a guide on how to configure custom labeling for

您可以在 LabelStudio 中修改默认的标注配置，以添加更多的目标标签，例如响应情感、相关性以及[其他类型的标注者反馈](https://labelstud.io/tags/)。

可以通过用户界面添加新的标注配置：转到 `Settings > Labeling Interface`，然后设置自定义配置，并添加额外的标签，例如用于情感的 `Choices` 或用于相关性的 `Rating`。请记住，任何配置中都必须包含 [`TextArea` 标签](https://labelstud.io/tags/textarea)，以便显示 LLM 的响应。

或者，您可以在项目创建之前的初始调用中指定标注配置：

In [ ]:
ls = LabelStudioCallbackHandler(
    project_config="""
<View>
<Text name="prompt" value="$prompt"/>
<TextArea name="response" toName="prompt"/>
<TextArea name="user_feedback" toName="prompt"/>
<Rating name="rating" toName="prompt"/>
<Choices name="sentiment" toName="prompt">
    <Choice value="Positive"/>
    <Choice value="Negative"/>
</Choices>
</View>
"""
)

请注意，如果项目不存在，它将根据指定的标签配置被创建。

## 其他参数

`LabelStudioCallbackHandler` 接受多个可选参数：

- **api_key** - Label Studio API 密钥。覆盖环境变量 `LABEL_STUDIO_API_KEY`。
- **url** - Label Studio URL。覆盖环境变量 `LABEL_STUDIO_URL`，默认值为 `http://localhost:8080`。
- **project_id** - 现有的 Label Studio 项目 ID。覆盖环境变量 `LABEL_STUDIO_PROJECT_ID`。数据将存储在此项目中。
- **project_name** - 如果未指定项目 ID，则使用此项目名称。将创建一个新项目。默认值为使用当前日期格式化后的 `"LangChain-%Y-%m-%d"`。
- **project_config** - [自定义标注配置](#custom-labeling-configuration)
- **mode**: 使用此快捷方式可以从头开始创建目标配置：
   - `"prompt"` - 单个提示，单个响应。为默认值。
   - `"chat"` - 多轮对话模式。